This notebook introduces Retrieval Augmented Generation (RAG) and demonstrates how to set up, ingest data into, and retrieve information from a vector database.

# Chatbots that know your data

Insert image of RAG

# Ingest into database

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

In [2]:
load_dotenv("configs.env")

True

In [3]:
# Folder with your .txt files
SOURCE_DIR = "data_files"
# Persistent vector database directory
DB_DIR = "vector_db"

In [4]:
loader = DirectoryLoader(SOURCE_DIR, glob="**/*.txt", show_progress=True)
docs = loader.load()

100%|██████████| 5/5 [00:02<00:00,  1.90it/s]


In [5]:
print(f"{len(docs)} txt files loaded successfully.")

5 txt files loaded successfully.


In [6]:
# split each book into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [7]:
print(f"{len(chunks)} chunks created successfully.")

3676 chunks created successfully.


In [8]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    persist_directory=DB_DIR,
)
print(f"✅ Ingestion complete. Persistent DB stored at: {DB_DIR}")

✅ Ingestion complete. Persistent DB stored at: vector_db


# Fetch from database

In [9]:
# === Load persistent vector DB ===
vectordb = Chroma(
    persist_directory=DB_DIR,
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"),
)

In [10]:
# retrieval using chromaDB
query = "Who is Irene Adler?"

In [11]:
# fetch top 3 most similar chunks
results = vectordb.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"\n-------------------Result {i}-------------------:")
    print(f"Source: {doc.metadata.get('source', 'unknown')}")
    # preview first 500 chars
    print(doc.page_content[:500])  


-------------------Result 1-------------------:
Source: data_files/The Adventures of Sherlock Holmes.txt
“And what of Irene Adler?” I asked.

“Oh, she has turned all the men’s heads down in that part. She is the daintiest thing under a bonnet on this planet. So say the Serpentine-mews, to a man. She lives quietly, sings at concerts, drives out at five every day, and returns at seven sharp for dinner. Seldom goes out at other times, except when she sings. Has only one male visitor, but a good deal of him. He is dark, handsome, and dashing, never calls less than once a day, and often twice. He is a M

-------------------Result 2-------------------:
Source: data_files/The Adventures of Sherlock Holmes.txt
“Then, pray consult,” said Holmes, shutting his eyes once more.

“The facts are briefly these: Some five years ago, during a lengthy visit to Warsaw, I made the acquaintance of the well-known adventuress, Irene Adler. The name is no doubt familiar to you.”

“Kindly look her up in my ind